## Monte Carlo Simulation using Geometric Brownian Motion
### By Hitaansh Gaur



In [1]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

We wil be considering a hypothetical stock market portfolio whose value follows Geometric Brownian Motion. We are assuming the continuously compounded real rate of interest is $r_\text{real}=0.87$% per year, and the continuously compounded inflation rate is $i = 2.87$% per year, the continuously compounded risk premium is $\pi = 6.26$% per year, and the annualized standard deviation of log returns is $\sigma = 15.91$%.

How did we get these numbers? They were obtained from the file "100 Year Summary" in the S&P Experiment Project where we did analysis on 100 years of both S&P and Bond data. 

Lets first see what the expected nominal value of one dollar will be after a certain number of years. We will compare this to the the real returns.